In [22]:
pwd

'/Users/vincenzo/CERM/Secondary Structure Predictor v0.1'

In [23]:
ls

SecStructPredictor.ipynb         prediction_flow.ipynb
Untitled.ipynb                   secondaryStructureClassifier.pt
Untitled1.ipynb                  struc_classifier2_0.7907


### Loading libraries

In [24]:
import torch
import os

### Setup

In [25]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [26]:
aminoacids = "ACDEFGHIKLMNPQRSTVWY"
struct_types = "hst-"

In [27]:
import os
os.listdir()

['prediction_flow.ipynb',
 'Untitled1.ipynb',
 'SecStructPredictor.ipynb',
 'Untitled.ipynb',
 'secondaryStructureClassifier.pt',
 'struc_classifier2_0.7907',
 '.ipynb_checkpoints']

### Model loading (deve essere nello stesso path del notebook...)

In [31]:
classifier = torch.load('struc_classifier2_0.7907', map_location=device)

ModuleNotFoundError: No module named 'secondary_structure'

### Show model architecture

In [25]:
classifier

RecurrentModel(
  (rnn_model): RNN(20, 20, num_layers=3)
  (final_layer): Linear(in_features=20, out_features=4, bias=True)
)

In [26]:
def sequenceToTensor(sequence):
    aminoacids = "ACDEFGHIKLMNPQRSTVWY"
    seqTensor = torch.zeros((len(sequence), len(aminoacids)))
    for j, aa in enumerate(sequence):
        aa_idx = aminoacids.index(aa)
        seqTensor[j, aa_idx] = 1.
    return seqTensor

### Usage: insert aminoacid sequence

In [36]:
aminoacid_sequence = input()

AYWTVTTTTTLMLMLMLMLMLLM


In [37]:
print(aminoacid_sequence)

AYWTVTTTTTLMLMLMLMLMLLM


### Data preprocessing step 

In [38]:
tensor_seq = sequenceToTensor(aminoacid_sequence)

### Prediction step

In [39]:
with torch.no_grad():
    classifier.eval()
    out, _ = classifier(tensor_seq.unsqueeze(1))
#print(out)

In [40]:
out_idxs = out.squeeze().argmax(1)
out_idxs

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Encoding the prediction in a readable format

In [43]:
predicted = [struct_types[j] for j in out_idxs]
print("aminoacids : ",list(aminoacid_sequence))
print("sec. struct: ", predicted)

aminoacids :  ['A', 'Y', 'W', 'T', 'V', 'T', 'T', 'T', 'T', 'T', 'L', 'M', 'L', 'M', 'L', 'M', 'L', 'M', 'L', 'M', 'L', 'L', 'M']
sec. struct:  ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h']
